# W&B WeaveによるOpenAI API監視

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />

<!--- @wandbcode{weave_openai_client_qs} -->

<a target="_blank" href="https://colab.research.google.com/github/wandb/weave/blob/master/examples/prompts/llm_monitoring/openai_client_quickstart.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**注意:** このcolabを実行するには[OpenAI APIキー](https://platform.openai.com/account/api-keys)が必要です。

W&B OpenAI統合を使用してOpenAI API呼び出しを監視し、プロジェクトやチームがLLMをどのように活用しているかを理解しましょう。
この例では、テンプレート化されたWeave Boardsを生成します：UIから探索およびカスタマイズできるLLM使用状況監視ダッシュボードです。

* LLM使用状況を自動的に追跡し、プロジェクト/チーム全体でコスト、レイテンシ、スループットなどの有用なメトリクスを集計
* すべてのOpenAI API呼び出しのログから動的にクエリを実行し、洞察を導出
* データをスライス、集計、探索するための視覚的な反復処理；興味深いパターンに焦点を当てるためのパネルのカスタマイズ；インタラクティブなダッシュボードを通じてチームとの進捗共有をより簡単に

<img src="https://raw.githubusercontent.com/wandb/weave/master/docs/assets/full_board_view.png">

[このWeave Boardのライブ版で遊んでみる →](http://wandb.me/llm-monitoring-board)

#### Weights & Biasesは初めてですか？[-> こちらでアカウントを登録 <-](https://wandb.ai/site)

# ステップ0: セットアップ

依存関係をインストールし、W&Bにログインして作業を保存・共有できるようにし、OpenAIで認証を行います。

In [ ]:
# if not already installed
!pip install -qqq weave openai tiktoken wandb

In [ ]:
import wandb
wandb.login()

In [ ]:
import weave
import os
WANDB_BASE_URL = "https://api.wandb.ai"
os.environ["WANDB_BASE_URL"] = WANDB_BASE_URL

In [ ]:
# authenticate with OpenAI
from getpass import getpass

if os.getenv("OPENAI_API_KEY") is None:
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

# ステップ1: W&Bでデータストリーミングとストレージを設定する

WB_ENTITYをあなたのwandbユーザー名またはチーム名に設定してください。W&Bにログインし、[wandb.ai/home](https://wandb.ai/home)のホームページに移動して、左サイドバーの「Profile」と「Teams」の下で有効なオプションを確認してください。

In [ ]:
WB_ENTITY = "" # set to your wandb username or team name
WB_PROJECT = "weave" # top-level directory for this work
STREAM_NAME = "openai_logs" # record table which stores the logs of OpenAI API calls as they stream in

# ステップ2: init_monitor()を呼び出す

OpenAI APIの使用状況の監視を開始するには、`init_monitor(<stream>)`を呼び出します。ここで`<stream>`は`<wandb_team_or_user>/<wandb_project>/<stream_name>`の形式になります。このストリームはすべてのOpenAI API呼び出しを記録・保存します。

このセルを実行すると、Weave UIで現在のプロジェクトを表示するためのリンクが出力されます。

In [ ]:
from weave.monitoring import openai, init_monitor
m = init_monitor(f"{WB_ENTITY}/{WB_PROJECT}/{STREAM_NAME}")

# specifying a single model for simplicity
OPENAI_MODEL = 'gpt-3.5-turbo'

# prefill with some sample logs
r = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": "hello world!"}])
r = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": "what is 2+2?"}])

# ステップ3: 監視ダッシュボードのプレビュー

上記のリンクをクリックしてデータストリームをプレビューし、右サイドバーの「OpenAI Monitor Board」をクリックして、このデータストリーム用のWeave Boardを作成します。

<img src="https://raw.githubusercontent.com/wandb/weave/master/docs/assets/short_board_attempt.gif" width=75%>

# ステップ4: LLM使用状況の探索と理解

作業を保存するには、ページ上部の自動生成された名前をクリックしてボードの名前を変更します。ボードを共有するには、右上の「Publish」をクリックします。

<img src="https://raw.githubusercontent.com/wandb/weave/master/docs/assets/publish_board_short.gif" width=75%>

反復作業中にリアルタイムで作業を可視化するには、以下のことができます：
* Boardを別のタブで開いたままにして、最新データを表示するためにリフレッシュする
* より簡単に参照できるようにBoardの名前を変更し、そのバージョンを「Publish」して他の人とリンクを共有する
* weave.wandb.aiから関連するW&BエンティティとW&Bプロジェクト名に移動して、以前に保存したBoardを見つける
* またはBoardテンプレートの新しいインスタンスを開いて、これまでに蓄積されたすべてのデータで新たに開始する

次に、OpenAI API呼び出しを追跡するいくつかの方法を説明します。使用ケースに応じてさらに多くの可能性があり、これらのスターターテンプレートから皆さんが作成するものを楽しみにしています。

# 例

## 例0: プロンプトとその補完をログに記録

ChatCompletionリクエストを監視し、対応するレスポンスを出力し、補完のテキストのみを抽出します。

In [ ]:
response = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[
        {"role": "user", "content": f"What is the meaning of life, the universe, and everything?"},
    ])
print(response['choices'][0]['message']['content'])

## 例1: 関連するパラメータを属性として追跡する

関心のあるパラメータを抽出し、ログ記録の属性として追跡します。
ここでは「システムプロンプト」を「プロンプトテンプレート」と「equation」パラメータから分離して追跡します。今回はChatCompletion呼び出しからの完全な構造化レスポンスを出力します。

In [ ]:
system_prompt = "you always write in bullet points"
prompt_template = 'solve the following equation step by step: {equation}'
params = {'equation': '4 * (3 - 1)'}
openai.ChatCompletion.create(model=OPENAI_MODEL,
                             messages=[
                                    {"role": "system", "content": system_prompt},
                                    {"role": "user", "content": prompt_template.format(**params)},
                                ],
                             # you can add additional attributes to the logged record
                             # see the monitor_api notebook for more examples
                             monitor_attributes={
                                 'system_prompt': system_prompt,
                                 'prompt_template': prompt_template,
                                 'params': params
                             })

## 例2: 継続的なメッセージストリームをログに記録する

メッセージのストリームを監視し、結果を単一のレコードとしてログに記録します。注意: このフォーマットではトークンはカウントされません。

In [ ]:
from weave.monitoring.openai import message_from_stream
r = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[
        {"role": "system", "content": "You are a robot and only speak in robot, like beep bloop bop."},
        {"role": "user", "content": "Tell me a 50-word story."},
    ], stream=True)
for s in message_from_stream(r):
    print(s, end='')

## 例3: プロンプトエンジニアリング実験の構造化

ここでは、システムプロンプト、ユーザーの質問、想定読者に対するいくつかの簡単なオプションを比較します。独自の実験を試して、Boardで探索し、異なるパラメータでグループ化する際に興味深い洞察が得られるかどうか確認してください。

In [ ]:
def explain_math(system_prompt, prompt_template, params):
    openai.ChatCompletion.create(model=OPENAI_MODEL,
                             messages=[
                                    {"role": "system", "content": system_prompt},
                                    {"role": "user", "content": prompt_template.format(**params)},
                                ],
                             # you can add additional attributes to the logged record
                             # see the monitor_api notebook for more examples
                             monitor_attributes={
                                 'system_prompt': system_prompt,
                                 'prompt_template': prompt_template,
                                 'params': params
                             })

In [ ]:
# feel free to substitute your own prompts :)
system_prompts = ["you're extremely flowery and poetic", "you're very direct and precise", "balance brevity with insight"]
prompt_template = 'explain the solution of the following to a {audience}: {equation}'
equations = ['x^2 + 4x + 9 = 0', '15 * (2 - 6) / 4']
audience = ["new student", "math genius"]

for system_prompt in system_prompts:
    for equation in equations:
        for person in audience:
            params = {"equation" : equation, "audience" : person}
            explain_math(system_prompt, prompt_template, params)